### Importing all the required libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import warnings
import matplotlib.pyplot as plt
imort seaborn as sns
warnings.filterwarnings('ignore')

In [5]:
#loading the dataset into a dataframe
df = pd.read_csv("../data/processed/feature_engineered_data_updated.csv")
df.head()

,Global_active_power,Global_reactive_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3,Hour,DayOfWeek,Is_Weekend,Rolling_mean_1h,Rolling_std_1h,Total_sub_metering,Unmetered_power
0,5.360,0.436,233.63,0.0,1.0,16.0,17,5,1,4.788000,0.808930,17.0,72.333333
1,5.374,0.498,233.29,0.0,2.0,17.0,17,5,1,4.983333,0.664567,19.0,70.566667
2,5.388,0.502,233.74,0.0,1.0,17.0,17,5,1,5.084500,0.579113,18.0,71.800000
3,3.666,0.528,235.68,0.0,1.0,17.0,17,5,1,4.800800,0.808676,18.0,43.100000
4,3.520,0.522,235.02,0.0,2.0,17.0,17,5,1,4.587333,0.892510,19.0,39.666667
